In [1]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
Neo4j_Pass = os.environ['NEO4J_PASS']

In [6]:
from neo4j import GraphDatabase

# Define Neo4j connection class
class ERPNeo4j:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()

    # Function to add a supplier node
    def add_supplier(self, supplier_id, name, country):
        with self.driver.session() as session:
            session.write_transaction(self._create_supplier, supplier_id, name, country)

    @staticmethod
    def _create_supplier(tx, supplier_id, name, country):
        query = """
        CREATE (s:Supplier {supplier_id: $supplier_id, name: $name, country: $country})
        RETURN s
        """
        result = tx.run(query, supplier_id=supplier_id, name=name, country=country)
        return result.single()

    # Function to add a product node
    def add_product(self, product_id, name, price):
        with self.driver.session() as session:
            session.write_transaction(self._create_product, product_id, name, price)

    @staticmethod
    def _create_product(tx, product_id, name, price):
        query = """
        CREATE (p:Product {product_id: $product_id, name: $name, price: $price})
        RETURN p
        """
        result = tx.run(query, product_id=product_id, name=name, price=price)
        return result.single()

    # Function to add a customer node
    def add_customer(self, customer_id, name, email):
        with self.driver.session() as session:
            session.write_transaction(self._create_customer, customer_id, name, email)

    @staticmethod
    def _create_customer(tx, customer_id, name, email):
        query = """
        CREATE (c:Customer {customer_id: $customer_id, name: $name, email: $email})
        RETURN c
        """
        result = tx.run(query, customer_id=customer_id, name=name, email=email)
        return result.single()

    # Function to create a supplier-product relationship
    def create_supply(self, supplier_id, product_id):
        with self.driver.session() as session:
            session.write_transaction(self._create_supply_rel, supplier_id, product_id)

    @staticmethod
    def _create_supply_rel(tx, supplier_id, product_id):
        query = """
        MATCH (s:Supplier {supplier_id: $supplier_id}), (p:Product {product_id: $product_id})
        CREATE (s)-[:SUPPLIES]->(p)
        RETURN s, p
        """
        result = tx.run(query, supplier_id=supplier_id, product_id=product_id)
        return result.single()

    # Function to create a customer-product relationship (purchase)
    def create_purchase(self, customer_id, product_id):
        with self.driver.session() as session:
            session.write_transaction(self._create_purchase_rel, customer_id, product_id)

    @staticmethod
    def _create_purchase_rel(tx, customer_id, product_id):
        query = """
        MATCH (c:Customer {customer_id: $customer_id}), (p:Product {product_id: $product_id})
        CREATE (c)-[:PURCHASED]->(p)
        RETURN c, p
        """
        result = tx.run(query, customer_id=customer_id, product_id=product_id)
        return result.single()

    # Query to find all products a supplier supplies
    def get_supplier_products(self, supplier_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_supplier_products, supplier_id)
            return result

    @staticmethod
    def _find_supplier_products(tx, supplier_id):
        query = """
        MATCH (s:Supplier {supplier_id: $supplier_id})-[:SUPPLIES]->(p:Product)
        RETURN p.name AS product, p.price AS price
        """
        result = tx.run(query, supplier_id=supplier_id)
        return [(record["product"], record["price"]) for record in result]

    # Query to find all purchases of a customer
    def get_customer_purchases(self, customer_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_customer_purchases, customer_id)
            return result

    @staticmethod
    def _find_customer_purchases(tx, customer_id):
        query = """
        MATCH (c:Customer {customer_id: $customer_id})-[:PURCHASED]->(p:Product)
        RETURN p.name AS product, p.price AS price
        """
        result = tx.run(query, customer_id=customer_id)
        return [(record["product"], record["price"]) for record in result]

    # Function to update supplier information
    def update_supplier(self, supplier_id, name=None, country=None):
        with self.driver.session() as session:
            session.write_transaction(self._update_supplier, supplier_id, name, country)

    @staticmethod
    def _update_supplier(tx, supplier_id, name, country):
        query = """
        MATCH (s:Supplier {supplier_id: $supplier_id})
        SET s.name = COALESCE($name, s.name), s.country = COALESCE($country, s.country)
        RETURN s
        """
        result = tx.run(query, supplier_id=supplier_id, name=name, country=country)
        return result.single()

    # Function to update product information
    def update_product(self, product_id, name=None, price=None):
        with self.driver.session() as session:
            session.write_transaction(self._update_product, product_id, name, price)

    @staticmethod
    def _update_product(tx, product_id, name, price):
        query = """
        MATCH (p:Product {product_id: $product_id})
        SET p.name = COALESCE($name, p.name), p.price = COALESCE($price, p.price)
        RETURN p
        """
        result = tx.run(query, product_id=product_id, name=name, price=price)
        return result.single()

    # Function to update customer information
    def update_customer(self, customer_id, name=None, email=None):
        with self.driver.session() as session:
            session.write_transaction(self._update_customer, customer_id, name, email)

    @staticmethod
    def _update_customer(tx, customer_id, name, email):
        query = """
        MATCH (c:Customer {customer_id: $customer_id})
        SET c.name = COALESCE($name, c.name), c.email = COALESCE($email, c.email)
        RETURN c
        """
        result = tx.run(query, customer_id=customer_id, name=name, email=email)
        return result.single()

    # Function to delete a supplier
    def delete_supplier(self, supplier_id):
        with self.driver.session() as session:
            session.write_transaction(self._delete_supplier, supplier_id)

    @staticmethod
    def _delete_supplier(tx, supplier_id):
        query = """
        MATCH (s:Supplier {supplier_id: $supplier_id})
        DETACH DELETE s
        """
        tx.run(query, supplier_id=supplier_id)

    # Function to delete a product
    def delete_product(self, product_id):
        with self.driver.session() as session:
            session.write_transaction(self._delete_product, product_id)

    @staticmethod
    def _delete_product(tx, product_id):
        query = """
        MATCH (p:Product {product_id: $product_id})
        DETACH DELETE p
        """
        tx.run(query, product_id=product_id)

    # Function to delete a customer
    def delete_customer(self, customer_id):
        with self.driver.session() as session:
            session.write_transaction(self._delete_customer, customer_id)

    @staticmethod
    def _delete_customer(tx, customer_id):
        query = """
        MATCH (c:Customer {customer_id: $customer_id})
        DETACH DELETE c
        """
        tx.run(query, customer_id=customer_id)

    # Query to find all suppliers of a product
    def get_product_suppliers(self, product_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_product_suppliers, product_id)
            return result

    @staticmethod
    def _find_product_suppliers(tx, product_id):
        query = """
        MATCH (s:Supplier)-[:SUPPLIES]->(p:Product {product_id: $product_id})
        RETURN s.name AS supplier, s.country AS country
        """
        result = tx.run(query, product_id=product_id)
        return [(record["supplier"], record["country"]) for record in result]

    # Query to find all customers who purchased a product
    def get_product_customers(self, product_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_product_customers, product_id)
            return result

    @staticmethod
    def _find_product_customers(tx, product_id):
        query = """
        MATCH (c:Customer)-[:PURCHASED]->(p:Product {product_id: $product_id})
        RETURN c.name AS customer, c.email AS email
        """
        result = tx.run(query, product_id=product_id)
        return [(record["customer"], record["email"]) for record in result]


# Example Usage
if __name__ == "__main__":
    # Connect to the Neo4j database
    erp = ERPNeo4j("bolt://localhost:7687", "neo4j", Neo4j_Pass)

    # Add suppliers
    erp.add_supplier("SUP001", "Acme Corp", "USA")
    erp.add_supplier("SUP002", "Global Goods", "China")

    # Add products
    erp.add_product("PROD001", "Laptop", 1200.00)
    erp.add_product("PROD002", "Smartphone", 800.00)

    # Add customers
    erp.add_customer("CUST001", "Alice Johnson", "alice@example.com")
    erp.add_customer("CUST002", "Bob Smith", "bob@example.com")

    # Create relationships (supplier supplies products)
    erp.create_supply("SUP001", "PROD001")
    erp.create_supply("SUP002", "PROD002")

    # Create relationships (customer purchases products)
    erp.create_purchase("CUST001", "PROD001")
    erp.create_purchase("CUST002", "PROD002")

    # Query products supplied by a supplier
    print("Products supplied by SUP001:")
    for product in erp.get_supplier_products("SUP001"):
        print(product)

    # Query purchases made by a customer
    print("\nPurchases made by CUST001:")
    for purchase in erp.get_customer_purchases("CUST001"):
        print(purchase)

    # Update supplier information
    erp.update_supplier("SUP001", name="Acme Corporation")

    # Update product information
    erp.update_product("PROD001", price=1100.00)

    # Update customer information
    erp.update_customer("CUST001", email="alice.johnson@example.com")

    # Delete a supplier
    erp.delete_supplier("SUP002")

    # Delete a product
    erp.delete_product("PROD002")

    # Delete a customer
    erp.delete_customer("CUST002")

    # Query suppliers of a product
    print("\nSuppliers of PROD001:")
    for supplier in erp.get_product_suppliers("PROD001"):
        print(supplier)

    # Query customers who purchased a product
    print("\nCustomers who purchased PROD001:")
    for customer in erp.get_product_customers("PROD001"):
        print(customer)

    # Close the connection
    erp.close()

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_supplier, supplier_id, name, country)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:28: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_product, product_id, name, price)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_customer, customer_id, name, email)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:56: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_supply_rel, supplier_id, product_id)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/392239

Products supplied by SUP001:


/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:86: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_supplier_products, supplier_id)


('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1200.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1200.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1200.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1200.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1100.0)
('Laptop', 1

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:101: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_customer_purchases, customer_id)


...


Suppliers of PROD001:


/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:200: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_product_suppliers, product_id)


('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation', 'USA')
('Acme Corporation',

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_56758/3922398067.py:215: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_product_customers, product_id)


('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('John Doe', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('John Doe', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('John Doe', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.johnson@example.com')
('Alice Johnson', 'alice.joh